In [1]:
# -*- coding: utf-8 -*-
"""

SEC Filing Scraper
@author: AdamGetbags

"""

# import modules
import requests
import pandas as pd

# create request header
headers = {'User-Agent': "email@address.com"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )




In [ ]:
from dataclasses import dataclass

@dataclass
class Transform:
    pass

In [2]:
import duckdb

In [1]:
from datetime import datetime
# review response / keys
print(companyTickers.json().keys())

# format response to dictionary and get first key/value
firstEntry = companyTickers.json()['0']

# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)
companyData['time_load'] = datetime.today()
# review data
print(companyData[:1])

cik = companyData[0:1].cik_str[0]




directCik

NameError: name 'companyTickers' is not defined

In [4]:
duckdb.sql('DESCRIBE SELECT * FROM companyData')# .fetchall()
# ('''

#     SELECT * FROM companyData
# ''')

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ cik_str     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ticker      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ time_load   │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [13]:
companyData.time_load   .str.len().max()

71

In [12]:
companyData.cik_str

0        0000789019
1        0000320193
2        0001045810
3        0001652044
4        0001018724
            ...    
10374    0001722438
10375    0001779372
10376    0000110095
10377    0001865782
10378    0001921158
Name: cik_str, Length: 10379, dtype: object

In [13]:
import sqlite3
connection = sqlite3.connect('2_ai.db'
                             )
companyData.to_sql(name='cik_tickers',con= connection,if_exists='append')

10379

In [14]:
# cik
direct_ticker = 'AAPL'
cik = str(companyData.query("ticker ==@direct_ticker ")['cik_str'].values[0])
cik

'0000320193'

In [15]:
# get company specific filing metadata
filingMetadata = requests.get(
    f'https://data.sec.gov/submissions/CIK{cik}.json',
    headers=headers
    )

# review json 
# print(filingMetadata.json().keys())
filingMetadata.json()['filings']
filingMetadata.json()['filings'].keys()
filingMetadata.json()['filings']['recent']
filingMetadata.json()['filings']['recent'].keys()

# dictionary to dataframe
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
             )

# review columns
allForms.columns
# allForms[['accessionNumber', 'reportDate', 'form']].head(50)

# allForms.query("form in ['10-Q','10-K']")

Index(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime',
       'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL',
       'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'],
      dtype='object')

In [8]:
filingMetadata.json().keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [9]:
allForms['filingDate'].min()

'2013-07-23'

In [16]:
# get company facts data
companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )

#review data
companyFacts.json().keys()
companyFacts.json()['facts']
companyFacts.json()['facts'].keys()

# filing metadata
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding'].keys()
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']['shares']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']['shares'][0]

# concept data // financial statement line items
companyFacts.json()['facts']['us-gaap']
companyFacts.json()['facts']['us-gaap'].keys()

# different amounts of data available per concept
companyFacts.json()['facts']['us-gaap']['AccountsPayable']
companyFacts.json()['facts']['us-gaap']['Revenues']
companyFacts.json()['facts']['us-gaap']['Assets']
companyFacts

<Response [200]>

In [17]:
companyFacts.json()['facts']['us-gaap'].keys()
companyFacts.json()['facts']['us-gaap']['Revenues'] # this for metabase
'label','description'
data = companyFacts.json()['facts']['us-gaap']['Revenues']['units']['USD']
pd.DataFrame(data)
# companyFacts.json()['facts']['us-gaap']['Revenues']['units'].keys()

,start,end,val,accn,fy,fp,form,filed,frame
0,2015-09-27,2016-09-24,215639000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016
1,2016-09-25,2016-12-31,78351000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016Q4
2,2017-01-01,2017-04-01,52896000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q1
3,2017-04-02,2017-07-01,45408000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q2
4,2016-09-25,2017-09-30,229234000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017
5,2017-07-02,2017-09-30,52579000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q3
6,2017-10-01,2017-12-30,88293000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q4
7,2017-12-31,2018-03-31,61137000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q1
8,2018-04-01,2018-06-30,53265000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q2
9,2017-10-01,2018-09-29,265595000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018


In [18]:
import re
tables_values = []
for index,elemnt in companyFacts.json()['facts']['us-gaap'].items():
    for measure, val, in elemnt['units'].items():
        tables_values.append([
        index,
        elemnt['label'],
        elemnt['description'],
        measure])
   




mmatch = re.search(r'\(.*?\)', tables_values[0][0])
if match:
    print(match.group(0).strip('()'))
else:
    print(tables_values[0][0])



# tables_values
# companyFacts.json()
companyFacts.json()
pd.DataFrame(tables_values)

AccountsPayable


,0,1,2,3
0,AccountsPayable,Accounts Payable (Deprecated 2009-01-31),Carrying value as of the balance sheet date of...,USD
1,AccountsPayableCurrent,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,USD
2,AccountsReceivableNetCurrent,"Accounts Receivable, after Allowance for Credi...","Amount, after allowance for credit loss, of ri...",USD
3,AccruedIncomeTaxesCurrent,"Accrued Income Taxes, Current",Carrying amount as of the balance sheet date o...,USD
4,AccruedIncomeTaxesNoncurrent,"Accrued Income Taxes, Noncurrent",Carrying amount as of the balance sheet date o...,USD
...,...,...,...,...
496,HedgedLiabilityFairValueHedge,"Hedged Liability, Fair Value Hedge",Amount of liability hedged in fair value hedgi...,USD
497,DeferredIncomeTaxAssetsNet,"Deferred Income Tax Assets, Net","Amount, after allocation of valuation allowanc...",USD
498,DeferredTaxLiabilitiesPropertyPlantAndEquipment,"Deferred Tax Liabilities, Property, Plant and ...",Amount of deferred tax liability attributable ...,USD
499,IncrementalCommonSharesAttributableToShareBase...,Incremental Common Shares Attributable to Dilu...,Additional shares included in the calculation ...,shares


In [19]:
# get company concept data
companyConcept = requests.get(
    (
    f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}'
     f'/us-gaap/AccruedLiabilities.json'
    ),
    headers=headers
    )

# review data
companyConcept.json().keys()
companyConcept.json()['units']
companyConcept.json()['units'].keys()
companyConcept.json()['units']['USD']
companyConcept.json()['units']['USD'][0]

# parse assets from single filing
companyConcept.json()['units']['USD'][0]['val']

# get all filings data 
assetsData = pd.DataFrame.from_dict((
               companyConcept.json()['units']['USD']))

# review data
assetsData.columns
assetsData.form

# get assets from 10Q forms and reset index
assets10Q = assetsData
assets10Q = assets10Q.reset_index(drop=True)

# plot 
assets10Q.plot(x='end', y='val')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [106]:
assetsData

,end,val,accn,fy,fp,form,filed,frame
0,2008-09-27,3719000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2008Q3I
1,2009-06-27,3338000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I


In [109]:
companyConcept.json()
companyConcept.json()['units']['USD'][0]['val']
# assetsData
companyConcept.json()

{'cik': 320193,
 'taxonomy': 'us-gaap',
 'tag': 'AccruedLiabilities',
 'label': 'Accrued Liabilities (Deprecated 2009-01-31)',
 'description': 'Carrying value as of the balance sheet date of obligations incurred and payable, pertaining to costs that are statutory in nature, are incurred on contractual obligations, or accumulate over time and for which invoices have not yet been received or will not be rendered. Examples include taxes, interest, rent and utilities. For classified balance sheets, used to reflect the current portion of the liabilities (due within one year or within the normal operating cycle if longer); for unclassified balance sheets, used to reflect the total liabilities (regardless of due date).',
 'entityName': 'Apple Inc.',
 'units': {'USD': [{'end': '2008-09-27',
    'val': 3719000000,
    'accn': '0001193125-09-153165',
    'fy': 2009,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2009-07-22',
    'frame': 'CY2008Q3I'},
   {'end': '2009-06-27',
    'val': 33380